<a href="https://colab.research.google.com/github/shreyanshtomar/moderation/blob/shreyansh_dev/server/notebooks/PyTorch/moderation_v1(resnet18).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting the data...

Source: https://www.kaggle.com/omeret/nsfw-nsafe & https://www.kaggle.com/omeret/nsfw-safe.

The combined dataset contains: 
* Number training images:  103518
* Num test images:  3365

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shreyanshtomar","key":"e573d8377370c57bf874be0ef0084247"}'}

In [ ]:
%%bash
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d omeret/nsfw-nsafe

100% 11.5G/11.5G [04:39<00:00, 49.3MB/s]
100% 11.5G/11.5G [04:39<00:00, 44.3MB/s]


In [ ]:
!unzip nsfw-nsafe.zip

In [ ]:
!kaggle datasets download -d omeret/nsfw-safe

100% 10.3G/10.4G [04:10<00:00, 34.4MB/s]
100% 10.4G/10.4G [04:10<00:00, 44.4MB/s]


In [ ]:
!unzip nsfw-safe.zip

##Importing Library and Data

To begin, import the torch and torchvision frameworks and their libraries with numpy, pandas, and sklearn. Libraries and functions used in the code below include:

* [transforms](https://pytorch.org/docs/stable/torchvision/transforms.html), for basic image transformation
* [torch.nn.functional](https://pytorch.org/docs/stable/nn.functional.html), which contains useful activation functions.
* [Dataset and Dataloader](https://pytorch.org/docs/0.3.0/torchvision/datasets.html), PyTorch's data loading utility 

---



In [ ]:
import copy
import cv2                
from glob import glob
from io import open
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os, os.path, random
from PIL import Image
import requests
import shutil
import time

import torch
from torch.autograd import Variable
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

plt.ion()   # interactive mode

%matplotlib inline

Specifying the data directories.

In [ ]:
data_dir = '/content/'
train_dir = os.path.join(data_dir, 'train/')
test_dir = os.path.join(data_dir, 'test/')

#Image Pre-processing

Images in a dataset do not usually have the same pixel intensity and dimensions.

You can stack multiple image transformation commands in [transform.Compose](https://pytorch.org/docs/master/torchvision/transforms.html#torchvision.transforms.Compose). Normalizing an image is an important step that makes model training stable and fast. In tranforms.Normalize() class, a list of means and standard deviations is sent in the form of a list. It uses this formula: ![alt text](https://i.imgur.com/pWSTFzG.png).
<br>
## Dataset Split

How well the model can learn depends on the variety and volume of the data. We need to divide our data into a training set and a validation set using PyTorch's [datasets.ImageFolder](https://pytorch.org/docs/master/torchvision/datasets.html?highlight=dataset%20image#torchvision.datasets.ImageFolder) utility since we already have downloaded the dataset in train_dir and test_dir directories.

**Training dataset**: The model learns from this dataset's examples. It fits a parameter to a classifier.

**Validation dataset**: The examples in the validation dataset are used to tune the hyperparameters, such as learning rate and epochs. The aim of creating a validation set is to avoid large overfitting of the model. It is a checkpoint to know if the model is fitted well with the training dataset.

Test dataset: This dataset test the final evolution of the model, measuring how well it has learned and predicted the desired output. It contains unseen, real-life data.



In [ ]:
data_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_data = datasets.ImageFolder(train_dir, transform=data_transform)
test_data = datasets.ImageFolder(test_dir, transform=data_transform)

# print out some data stats
print('Num training images: ', len(train_data))
print('Num test images: ', len(test_data))

Num training images:  103518
Num test images:  3365


Whenever you initialize the batch of images, it is on the CPU for computation by default. The function torch.cuda.is_available() will check whether a GPU is present. If CUDA is present, .device("cuda") will route the tensor to the GPU for computation.

The device will use CUDA with a single GPU processor. This will make our calculations faster. If you have a CPU in your system, no problem. You can use Google Colab, which provides free GPU.

In [ ]:
# check if CUDA is available
device_avail = torch.cuda.is_available()

if not device:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...



In the code below, `dataloader` combines a dataset and a sampler and provides an iterable over the given dataset. `dataset()` indicates which dataset to load form the available data. For details, [read this documentation](https://pytorch.org/docs/stable/data.html#module-torch.utils.data).


In [ ]:
# define dataloader parameters
batch_size = 128
num_workers=0

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)
loaders_scratch = {
    'train': train_loader,
    'test': test_loader
}

In [ ]:
# Visualize some sample data
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))

In [ ]:
images, labels = next(iter(train_loader)) 
print("images-size:", images.shape)

out = torchvision.utils.make_grid(images)
print("out-size:", out.shape)

images-size: torch.Size([128, 3, 224, 224])
out-size: torch.Size([3, 3618, 1810])


#Transfer Learning with Pytorch

The main aim of transfer learning (TL) is to implement a model quickly. To solve the current problem, instead of creating a DNN (dense neural network) from scratch, the model will transfer the features it has learned from the different dataset that has performed the same task. This transaction is also known as knowledge transfer.

<img src="https://i.imgur.com/3sx8Y3i.png" width = "600"/>

Why Resnet-18 ? 

ResNet-18 is a convolutional neural network that is trained on more than a million images from the ImageNet database. There are 18 layers present in its architecture. It is `very useful and efficient in image classification` and can classify images into 1000 object categories. The network has an image input size of 224x224.
Also,
1. Networks with large number (even thousands) of layers can be trained easily without increasing the training error percentage.
2. ResNets help in tackling the vanishing gradient problem using identity mapping.

The Pytorch API calls a pre-trained model of ResNet18 by using models.resnet18(pretrained=True), the function from TorchVision's model library. ResNet-18 architecture is described below.
<img src="https://i.imgur.com/XwcnU5x.png" width = "600"/>

In [ ]:
net = models.resnet18(pretrained=True)
net = net.cuda() if device else net
net

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Specifying Loss & Optimizer:

Loss function and optimization go hand-in-hand. Loss function checks whether the model is moving in the correct direction and making progress, whereas optimization improves the model to deliver accurate results.
Select any one optimizer algorithm available in the [torch.optim](https://pytorch.org/docs/master/optim.html) package. The optimizers have some elements of the gradient descent.

Also, setting the hyperparameters below.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 128)
net.fc = net.fc.cuda() if device else net.fc

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
n_epochs = 5
print_every = 10
valid_loss_min = np.Inf
val_loss = [] 
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader)

    ###################
    # train the model #
    ###################
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(train_loader):
        data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad() # clear-the-gradients-of-all-optimized-variables
        
        outputs = net(data_) # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        loss = criterion(outputs, target_) # calculate-the-batch-loss
        loss.backward()  # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        optimizer.step()  # perform-a-ingle-optimization-step (parameter-update)

        running_loss += loss.item() # update-training-loss
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0

    ######################    
    # validate the model #
    ######################
    with torch.no_grad():
        net.eval()
        for data_t, target_t in (test_loader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t) # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
            loss_t = criterion(outputs_t, target_t) # calculate-the-batch-loss
            batch_loss += loss_t.item()  # update-average-validation-loss 
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(test_loader)) 
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')

        
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    net.train()


Epoch 1

Epoch [1/5], Step [0/809], Loss: 5.2429
Epoch [1/5], Step [20/809], Loss: 5.2985
Epoch [1/5], Step [40/809], Loss: 5.2799
Epoch [1/5], Step [60/809], Loss: 5.1616
Epoch [1/5], Step [80/809], Loss: 5.2010
Epoch [1/5], Step [100/809], Loss: 4.9265
Epoch [1/5], Step [120/809], Loss: 4.7991
Epoch [1/5], Step [140/809], Loss: 4.7754
Epoch [1/5], Step [160/809], Loss: 4.6035
Epoch [1/5], Step [180/809], Loss: 4.6426
Epoch [1/5], Step [200/809], Loss: 4.4863
Epoch [1/5], Step [220/809], Loss: 4.3222
Epoch [1/5], Step [240/809], Loss: 4.3232
Epoch [1/5], Step [260/809], Loss: 4.2281
Epoch [1/5], Step [280/809], Loss: 3.9456
Epoch [1/5], Step [300/809], Loss: 3.9685
Epoch [1/5], Step [320/809], Loss: 3.8650
Epoch [1/5], Step [340/809], Loss: 3.8362
Epoch [1/5], Step [360/809], Loss: 3.5822
Epoch [1/5], Step [380/809], Loss: 3.3608
Epoch [1/5], Step [400/809], Loss: 3.5758
Epoch [1/5], Step [420/809], Loss: 3.2884
Epoch [1/5], Step [440/809], Loss: 3.1906
Epoch [1/5], Step [460/809], Lo

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch [1/5], Step [500/809], Loss: 2.9568
Epoch [1/5], Step [520/809], Loss: 2.6109
Epoch [1/5], Step [540/809], Loss: 2.8598
Epoch [1/5], Step [560/809], Loss: 2.2944
Epoch [1/5], Step [580/809], Loss: 2.6249
Epoch [1/5], Step [600/809], Loss: 2.5698
Epoch [1/5], Step [620/809], Loss: 2.2766
Epoch [1/5], Step [640/809], Loss: 2.2526
Epoch [1/5], Step [660/809], Loss: 2.2072
Epoch [1/5], Step [680/809], Loss: 2.1748
Epoch [1/5], Step [700/809], Loss: 2.0582
Epoch [1/5], Step [720/809], Loss: 2.3753
Epoch [1/5], Step [740/809], Loss: 1.8946
Epoch [1/5], Step [760/809], Loss: 1.7379
Epoch [1/5], Step [780/809], Loss: 1.7760
Epoch [1/5], Step [800/809], Loss: 1.7112

train-loss: 3.4795, train-acc: 47.8004
validation loss: 1.8241, validation acc: 86.9242

Improvement-Detected, save-model
Epoch 2

Epoch [2/5], Step [0/809], Loss: 1.6534
Epoch [2/5], Step [20/809], Loss: 2.0337
Epoch [2/5], Step [40/809], Loss: 1.6541
Epoch [2/5], Step [60/809], Loss: 1.8867
Epoch [2/5], Step [80/809], Loss:

NameError: ignored

In [ ]:
def test(loaders, model, criterion, train_on_gpu):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)

    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, net, criterion, device)

Test Loss: 0.645461


Test Accuracy: 91% (3072/3365)


In [ ]:
from PIL import Image
import torchvision.transforms as transforms

def load_input_image(img_path):    
    image = Image.open(img_path).convert('RGB')
    prediction_transform = transforms.Compose([transforms.Resize(size=(224, 224)),
                                     transforms.ToTensor(), 
                                     standard_normalization])

    # discard the transparent, alpha channel (that's the :3) and add the batch dimension
    image = prediction_transform(image)[:3,:,:].unsqueeze(0)
    return image

In [ ]:
loaders_transfer = loaders_scratch.copy()

In [ ]:
class_names = [item[4:].replace("_", " ") for item in loaders_transfer['train'].dataset.classes]


def predict_nsfw(model, class_names, img_path):
    # load the image and return the predicted breed
    img = load_input_image(img_path)
    model = model.cpu()
    model.eval()
    idx = torch.argmax(model(img))
    return class_names[idx]

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

In [ ]:
torch.save(net.state_dict(), 'resnet18_checkpoint.pth')

In [ ]:
net.load_state_dict(torch.load('resnet18_checkpoint.pth'))

<All keys matched successfully>

##Preparing the image for inference


In [ ]:
import io

import torchvision.transforms as transforms
from PIL import Image

def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.5, 0.5, 0.5],
                                            [0.5, 0.5, 0.5])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)

*The above method takes image data in bytes, applies the series of transforms and returns a tensor.*

Testing the above method..

In [ ]:
with open("/content/park.jpeg", 'rb') as f:
    image_bytes = f.read()
    tensor = transform_image(image_bytes=image_bytes)
    print(tensor)

#Inference

In [ ]:
!cp '/content/drive/My Drive/fastai_chkpts/resnet18_checkpoint.pth' '/content/' # Getting checkpoint in the ./content directory

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
# check if CUDA is available
device_avail = torch.cuda.is_available()

if not device_avail:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is not available.  Training on CPU ...


In [ ]:
net = models.resnet18(pretrained=True)
#net = net.cuda() if device_avail else net.cpu()
net = net.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 128)
net.fc = net.fc.to(device)

In [ ]:
if not device_avail:
    net.load_state_dict(torch.load('/content/resnet18_checkpoint.pth', map_location=torch.device('cpu')))
else:
    net.load_state_dict(torch.load('/content/resnet18_checkpoint.pth'))
net.eval()

In [ ]:
def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    tensor = tensor.to(device) #Moving tensor to the available device
    outputs = net.forward(tensor)
    _, y_hat = outputs.max(1)
    return y_hat

In [ ]:
with open("/content/park.jpeg", 'rb') as f:
    image_bytes = f.read()
    print(get_prediction(image_bytes=image_bytes))

tensor([1])


In [ ]:
with open("/content/test.jpg", 'rb') as f:
    image_bytes = f.read()
    print(get_prediction(image_bytes=image_bytes))

tensor([0])


*The tensor y_hat will contain the index of the predicted class id. However, we need a human readable class name. For that we need a class id to name.*

**Therefore, writing the get_prediction function again...**

In [ ]:
class_index = {0: 'nsfw', 1: 'sfw'}

def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes).to(device)
    outputs = net.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = y_hat.item()
    return class_index[predicted_idx]

In [ ]:
%time
with open("/content/park.jpeg", 'rb') as f:
    image_bytes = f.read()
    print(get_prediction(image_bytes=image_bytes))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs
sfw


In [ ]:
%time
with open("/content/test.jpg", 'rb') as f:
    image_bytes = f.read()
    print(get_prediction(image_bytes=image_bytes))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.29 µs
nsfw
